### 라이브러리

In [1]:
import os
import shutil
from dotenv import load_dotenv

from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

### API 설정

In [2]:
# 🔐 API 키 불러오기
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

### PDF 로드 / 임베딩 / DB생성

In [ ]:
    # 📁 카테고리별 PDF 파일 경로
    category_pdf_map = {"cancer"  : "KB 실버암 간편건강보험Plus.pdf",
                        "medical" : "KB개인실손의료비보험.pdf",
                        "accident": "KB개인상해보험.pdf",
                        "fire"    : "KB아파트화재보험.pdf",
                        }

    # 💾 벡터스토어 저장 루트
    VECTOR_ROOT = "vectorstores"

    # 📑 텍스트 분할기
    text_splitter = RecursiveCharacterTextSplitter(chunk_size    = 200, 
                                                chunk_overlap = 20,
                                                )

    # 🧠 OpenAI 임베딩
    embedding = OpenAIEmbeddings(model="text-embedding-3-small", 
                                openai_api_key=OPENAI_API_KEY,
                                )

    # ✅ (최초 1회 실행) PDF → 벡터스토어 생성 및 저장
    def build_vectorstores():
        for category, pdf_path in category_pdf_map.items():
            print(f"[+] Building vectorstore for category: {category}")
            loader = PyPDFLoader(pdf_path)
            pages = loader.load_and_split()
            docs = text_splitter.split_documents(pages)

            vectorstore_path = os.path.join(VECTOR_ROOT, category)
            if os.path.exists(vectorstore_path):
                shutil.rmtree(vectorstore_path)
            vectorstore = FAISS.from_documents(docs, embedding=embedding)
            vectorstore.save_local(vectorstore_path)

    # 🧠 카테고리별 키워드 기반 매핑
    category_keywords = {
        "cancer": [
            "암", "유사암", "특정암", "전이암", "재진단암", "갑상선암", "폐암", "간암", "췌장암",
            "소화기관암", "혈액암", "생식기암", "항암", "항암치료", "방사선치료", "항암방사선",
            "항암약물", "표적항암", "호르몬약물", "CAR-T", "진단비", "암진단비", "암사망"
        ],
        "medical": [
            "실손", "의료비", "입원", "통원", "진료비", "검사비", "수술", "응급실", "치료",
            "자기부담금", "보험금 한도", "진단서", "의무기록", "보상종목", "다수보험", "연대책임"
        ],
        "accident": [
            "상해", "재해", "사고", "교통사고", "골절", "화상", "후유장해", "사고사망",
            "입원비", "수술비", "상해사망", "상해특약"
        ],
        "fire": [
            "화재", "폭발", "붕괴", "누수", "도난", "배상책임", "재산", "가재도구",
            "복구", "손해", "피해", "주택", "화재보험", "화재사고"
        ]
    }

    # 📌 질문 → 카테고리 매핑 함수
    def classify_question(question):
        for category, keywords in category_keywords.items():
            if any(keyword in question for keyword in keywords):
                return category
        return "cancer"  # 기본 fallback


C:\Users\KDT2\AppData\Local\Temp\ipykernel_74932\1492735552.py:17: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model="text-embedding-3-small",


### 모델

In [4]:
# 💬 LLM 및 RAG 설정
llm = ChatAnthropic(model = "claude-opus-4-20250514", 
                    temperature = 0, 
                    max_tokens = 1024, 
                    api_key = ANTHROPIC_API_KEY,
                    )
prompt = hub.pull("rlm/rag-prompt")
llm_only_chain = llm | StrOutputParser()

# 📣 질문 → 응답 생성 함수
def answer_question(question):
    category = classify_question(question)
    vectorstore_path = os.path.join(VECTOR_ROOT, category)
    print(f"\n[📂 선택된 카테고리: {category}]")

    # 벡터스토어 로드 및 Retriever 생성
    vectorstore = FAISS.load_local(vectorstore_path,
                                   embeddings = embedding,
                                   allow_dangerous_deserialization = True  # 추가
                                   )
    
    retriever = vectorstore.as_retriever()

    # RAG 체인 생성
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 출력
    print("🧾 질문:", question)
    print("\n💬 [LLM 단독 응답]:")
    print(llm_only_chain.invoke(question))
    print("\n📚 [RAG 기반 응답]:")
    print(rag_chain.invoke(question))

C:\Users\KDT2\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


### 벡터스토어 생성

In [5]:
# ✅ 최초 실행 시만 벡터스토어 생성
build_vectorstores()

[+] Building vectorstore for category: cancer
[+] Building vectorstore for category: medical
[+] Building vectorstore for category: accident
[+] Building vectorstore for category: fire


### 실행

In [7]:
# 📌 실행 예시
if __name__ == "__main__":
    # ✅ 최초 실행 시만 벡터스토어 생성
    # build_vectorstores()

    # 질문 예시
    # question = "유사암진단비iii 를 청구하려고 하는데 갑상선암은 어떤 검사를 통해 진단받아야돼?" 
    question = "통합전이암진단비의 보장범위는 어떻게 돼?"   
    answer_question(question)


[📂 선택된 카테고리: cancer]
🧾 질문: 통합전이암진단비의 보장범위는 어떻게 돼?

💬 [LLM 단독 응답]:
통합전이암진단비의 일반적인 보장범위를 설명드리겠습니다.

## 주요 보장내용

**1. 전이암 진단 시**
- 원발암이 다른 장기로 전이된 경우
- 최초 1회에 한해 진단비 지급
- 조직검사 등으로 확진된 경우

**2. 보장 제외 사항**
- 원발암 자체는 보장하지 않음
- 재발암은 일반적으로 제외
- 전암성 병변(상피내암 등) 제외

**3. 주요 특징**
- 원발암 진단 후 전이암 발생 시 추가 보장
- 보험사별로 보장금액 차이 있음
- 면책기간/감액기간 적용 가능

## 확인 필요사항

보험상품마다 세부 약관이 다르므로:
- 가입하신 보험약관 확인
- 보장 개시일 확인
- 진단비 지급 조건 상세 확인

정확한 보장범위는 가입하신 보험사에 직접 문의하시거나 약관을 확인하시는 것이 가장 정확합니다.

📚 [RAG 기반 응답]:
제공된 문서에는 "통합전이암진단비"라는 명칭의 보장은 나타나지 않습니다. 다만 11대특정암진단비(전이암포함), 폐암진단비(전이암포함), 간암·담낭암·기타담도암 및 췌장암진단비(전이암포함), 3대특정고액암진단비(전이암포함), 소화기관암진단비(전이암포함) 등 개별 암종별로 전이암을 포함한 진단비가 보장되는 것으로 확인됩니다. 각 보장은 보험가입금액의 50%에서 100%까지 지급되는 것으로 나타납니다.
